In [2]:
import pandas as pd
import glob

In [47]:
path = "revised reviews"
dfs = []
for file in glob.glob(path + '/*.xlsx'):
    dfs.append(pd.read_excel(file))
    
df = pd.concat(dfs)


df.columns = df.columns.str.strip()
#print(df.columns)

#cleaning up and creating Year
df.rename(columns={
    'Who are you': 'Rater',
    'Nature (Simon) see classification details': 'Nature',
    'Context (Simon) see classification details. Use "other" if necessary': 'Context',
    'Theme/topic (Simon) see classification details. Use "other" if necessary': 'Theme',
    'Scope (Simon) see classification details': 'Scope',
    'Title of paper (for cross-checking only)': 'Title',
    'UK institutions contributing data/context, select at least one ("no data/context" is an option, only use if there is absolutely nothing: no module titles, no description of year groups etc). Some institutions are titled "University Of..."': 'Institutions'
}, inplace=True)

df['Year'] = df['Which paper'].astype(str).str[:4]

df = df.drop(columns=['Timestamp',
                      'Rater',
                      'Are you primary or secondary reviewer',
                      'Author emails. One per line',
                      'Section headings, top-level only. One per line',
                      'Number of references in the paper',
                      'Institutions',
                      'C'
                     ])

df

,Which paper,Title,Nature,Scope,Context,Theme,revision comments,Notes and queries,Year
0,2025_3702212.3702213,Learning Programming Languages by Pantomime,Report,Program/department,Programming,teaching/learning techniques,NaN,NaN,2025
1,2025_3702212.3702213,Learning Programming Languages by Pantomime,Report,Program/department,Programming,teaching/learning techniques,NaN,NaN,2025
2,2025_3702212.3702214,Learning without Limits: Analysing the Usage o...,Report,Subject,software engineering,generative AI,NaN,NaN,2025
3,2025_3702212.3702214,Learning without Limits: Analysing the Usage o...,Report,Subject,software engineering,generative AI,NaN,NaN,2025
4,2025_3702212.3702215,Group Assignments and Support Aimed to Develop...,Report,Subject,groupwork,peer learning/evaluation/feedback,new category,NaN,2025
...,...,...,...,...,...,...,...,...,...
62,2022_3498343.3498350,Application of AmazonWeb Services within teach...,Report,Program/department,cloud computing,curriculum,NaN,NaN,2022
63,2022_3498343.3498351,Assessing Knowledge and Skills in Foresnics wi...,Report,Subject,security,assessment techniques,NaN,NaN,2022
64,2022_3498343.3498351,Assessing Knowledge and Skills in Forensics wi...,Report,Subject,security,assessment techniques,NaN,"subject - selected security, but perhaps foren...",2022
65,2022_3498343.3498353,LMC+ Scratch: A recipe to construct a mental m...,Report,Subject,Programming,teaching/learning techniques,NaN,There are CEP issues hidden in here - why they...,2022


In [76]:
originalOptions = {"Context": 
                         ["artificial intelligence", "broad-based", "capstone project", "communication skills", "compilers", "cryptography", "data mining", "database", "data structures",  "eBusiness/eCommerce", "ethics/professionalism", "formal methods", "graphics", "groupwork", "hardware/architecture", "human-computer interaction", "information systems", "introduction to IT", "literature",  "logic", "mathematics", "multimedia", "networks", "operating systems", "postgraduate/research", "programming languages", "Programming", "robotics", "school/outreach", "security", "software engineering", "study planning", "systems analysis", "webpage development", "work experience"],
                  "Theme":
                      ["ability/aptitude/understanding", "accessibility", "assessment techniques", "assessment tools", "attitudes", "cheating & plagiarism", "competitions", "credit for prior learning", "curriculum", "educational technology", "ethics/professional issues", "gender issues", "language/culture issues", "online/distance delivery", "recruitment, progression,", "pathways", "research", "teachers", "teaching/learning techniques", "teaching/learning theories & models", "teaching/learning tools", "tutors, demonstrators, mentors"
]
}

newOptions = {"Context": ['service teaching', 'cloud computing'],
            "Theme": ['generative AI', 'peer learning/evaluation/feedback'],
}

allOptions = originalOptions

for key, val in newOptions.items():
    allOptions[key] = allOptions[key] + val


allOptions

{'Context': ['artificial intelligence',
  'broad-based',
  'capstone project',
  'communication skills',
  'compilers',
  'cryptography',
  'data mining',
  'database',
  'data structures',
  'eBusiness/eCommerce',
  'ethics/professionalism',
  'formal methods',
  'graphics',
  'groupwork',
  'hardware/architecture',
  'human-computer interaction',
  'information systems',
  'introduction to IT',
  'literature',
  'logic',
  'mathematics',
  'multimedia',
  'networks',
  'operating systems',
  'postgraduate/research',
  'programming languages',
  'Programming',
  'robotics',
  'school/outreach',
  'security',
  'software engineering',
  'study planning',
  'systems analysis',
  'webpage development',
  'work experience',
  'service teaching',
  'cloud computing'],
 'Theme': ['ability/aptitude/understanding',
  'accessibility',
  'assessment techniques',
  'assessment tools',
  'attitudes',
  'cheating & plagiarism',
  'competitions',
  'credit for prior learning',
  'curriculum',
  'ed

In [78]:
papers = df['Which paper'].unique()

problems = {}
for paper in papers:
    paper_problems = {}
    paper_df = df[df['Which paper'] == paper]
    
    review_notes = list(paper_df['revision comments'].dropna().unique())
    
    if len(paper_df) !=2:
        paper_problems['count'] = len(paper_df)
        
    for col_should_be_equal in ['Nature', 'Scope', 'Context', 'Theme']:
        uvs = list(paper_df[col_should_be_equal].unique())
        if len(uvs)> 1 and (not ('keep both' in review_notes)):
            paper_problems[col_should_be_equal + "_mismatch"] = uvs
            
    for col, options in allOptions.items():
        uvs = list(paper_df[col].unique())
        if not (all(x in options for x in uvs)):
            paper_problems[col+"_option"] = uvs        
    
    if len(paper_problems)>0:
        paper_problems['notes'] = review_notes 
        problems[paper] = paper_problems
        
problems

{'2023_3573260.3573261': {'Scope_mismatch': ['Program/department', 'Subject'],
  'notes': []},
 '2023_3573260.3573265': {'Context_option': ['data science'],
  'notes': ['new option', 'keep both']},
 '2024_3633053.3633055': {'Context_option': ['outreach not schools'],
  'notes': []},
 '2024_3633053.3633058': {'Context_option': ['outreach not schools'],
  'notes': []},
 '2024_3633053.3633061': {'Scope_mismatch': ['Program/department', 'Subject'],
  'notes': []},
 '2020_3372356.3372358': {'count': 1, 'notes': []},
 '2020_3372356.3372359': {'count': 1, 'notes': []},
 '2020_3372356.3372362': {'Theme_option': ['accreditation'], 'notes': []},
 '2020_3372356.3372366': {'Theme_option': ['inclusion'],
  'notes': ['new category (broadening of gender_']},
 '2021_3437914.3437971': {'count': 1, 'notes': []}}